In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

In [ ]:
import os
import zipfile

In [ ]:
os.listdir()

In [ ]:
file = 'cell-images-for-detecting-malaria.zip'
zip_ref = zipfile.ZipFile(file, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
dir_name = './cell_images/cell_images'
os.listdir(dir_name)

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# ImageDataGenerator class provides the abilities to process image data efficiently
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.random.set_seed(12345)

In [ ]:
# Define the Dataset Generator. We will use this to read files. Validation split
# determines the split into training and testing
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split = 0.2,
                                   horizontal_flip = True)

In [ ]:
# Read the training set
training_set = train_datagen.flow_from_directory(dir_name,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 subset='training',
                                                 class_mode = 'binary')

In [ ]:
validation_set = train_datagen.flow_from_directory(dir_name,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 subset='validation',
                                                 class_mode = 'binary')

In [ ]:
print(dir_name)

In [ ]:
training_set.class_indices, validation_set.class_indices

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D,Flatten, Dense

In [ ]:
model = Sequential() # Create a blank model

In [ ]:
# Add the first Convolution Layer and Max Pool layer
model.add(Conv2D(name='cnv_layer',
                 filters=32,
                 kernel_size=3,
                 activation='relu',
                 padding='same',
                 input_shape=[224, 224, 3]))

In [ ]:
model.add(MaxPool2D(pool_size = 2)) # Maxpool Layer

In [ ]:
model.add(Flatten(name='flatten')) # Add the flatten layer

In [ ]:
model.add(Dense(units = 64,
                name='dense',
                activation = 'relu')) #Add the dense layer

In [ ]:
model.add(Dense(name='output',
                units=1,
                activation = 'sigmoid')) # Final Output layer

In [ ]:
model.summary()

In [ ]:
(3*3*3+1)*32

In [ ]:
(3*3*3+1)*32, 112*112*32, (401408+1)*64

In [ ]:
# Compile the model
model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.fit(training_set,
          validation_data = validation_set,
          epochs=3) # Train the model for 3 epochs

# Transfer Learning

In [ ]:
from tensorflow.keras.applications import MobileNetV2

In [ ]:
mobile_net = MobileNetV2(input_shape=(224, 224, 3), include_top=False)
mobile_net.trainable = False

In [ ]:
trf_model = Sequential()

In [ ]:
trf_model.add(mobile_net)

In [ ]:
trf_model.add(Flatten())
trf_model.add(Dense(256, activation = 'relu'))

In [ ]:
trf_model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
trf_model.summary()

In [ ]:
# Compile the model
trf_model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
trf_model.fit(training_set,
              validation_data = validation_set,
              epochs = 3)